In [ ]:
%pip install -U transformers bitsandbytes accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Upstage/SOLAR-10.7B-Instruct-v1.0")
model = AutoModelForCausalLM.from_pretrained(
    "Upstage/SOLAR-10.7B-Instruct-v1.0",
    device_map="auto",
    load_in_4bit=True,
)

In [ ]:
conversation = [ {'role': 'user', 'content': 'Hello?'} ]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, use_cache=True, max_length=4096)
output_text = tokenizer.decode(outputs[0])
print(output_text)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

class ChatApp:
    def __init__(self):
        self.message_history = []

        # Create UI components
        self.output = widgets.Output(layout={'border': 'None'})
        self.input_box = widgets.Text(placeholder='Type your message...', layout={'width': '70%'})
        self.input_box.on_submit(self.send_message)
        self.send_button = widgets.Button(description='Send', layout={'width': '20%'})
        self.send_button.on_click(self.send_message)

        # Display UI
        display(widgets.HBox([self.input_box, self.send_button]))
        display(self.output)

    def send_message(self, _):
        message = self.input_box.value
        self.input_box.value = ''  # Clear the input box

        # Display sent message
        with self.output:
            display(Markdown(f'### User:\n{message}'))
            self.message_history.append({'role': 'user', 'content': message})

        # Simulate a response (you can replace this with actual logic)
        response = self.generate_response(message)

        # Display received message
        with self.output:
            text = response.replace("<s>", "").replace("</s>","").split("### Assistant:")[-1]
            display(Markdown(f'\n### Assistant: \n{text}'))
            self.message_history.append({'role': 'assistant', 'content': text})

    def generate_response(self, message):
        progress_bar = widgets.IntProgress(
            value=0,
            min=0,
            max=100,
            bar_style='', # 'success', 'info', 'warning', 'danger' or ''
            orientation='horizontal'
        )
        display(progress_bar)
        # Replace this with your own logic to generate a response
        prompt = tokenizer.apply_chat_template(self.message_history, tokenize=False, add_generation_prompt=True)
        progress_bar.value += 5
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        progress_bar.value += 15
        outputs = model.generate(**inputs, use_cache=True, max_length=4096)
        progress_bar.value += 30
        output_text = tokenizer.decode(outputs[0])
        progress_bar.value += 50
        progress_bar.bar_style = "success"
        progress_bar.close()
        return output_text

# Create an instance of the ChatApp
chat_app = ChatApp()